In [1]:
import os
%pwd

'c:\\Users\\rohan\\OneDrive\\Desktop\\New folder\\RetailDemandForecast\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\rohan\\OneDrive\\Desktop\\New folder\\RetailDemandForecast'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataInjestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path
    final_dataset_dir:Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    seasonality_mode:str
    daily_seasonality:bool
    weekly_seasonality:bool
    yearly_seasonality:bool

@dataclass(frozen=True)
class ModelTrainingConfig:
  root_dir: Path
  trained_model_path: Path
  dataset_path: Path
  base_model_path:Path
  store_column:str='Store'
  date_column: str = 'Date'
  target_column: str = 'Weekly_Sales_x'
  train_split: int = 120

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    evaluation_results_path: Path
    dataset_path: Path
    model_path: Path
    train_split: int = 120
    forecast_periods: int = 23
    frequency: str='W-Fri'
    store_column:str='Store'
    date_column: str = 'Date'
    target_column: str = 'Weekly_Sales_x'

@dataclass
class ModelPredictionConfig:
    root_dir: Path
    dataset_path: Path
    model_path: Path
    

In [4]:
from src.retailDemand.constants import *
from src.retailDemand.utils import *

In [5]:
%pwd

'c:\\Users\\rohan\\OneDrive\\Desktop\\New folder\\RetailDemandForecast'

In [6]:
from src.retailDemand.utils.common import createDir, read_yaml


class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        createDir([self.config.artifacts_root])

    def getDataInjestionConfig(self)->DataInjestionConfig:
        config=self.config.data_injestion

        createDir([config.root_dir])
        data_injestion_config=DataInjestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            source_url=config.source_url,
            final_dataset_dir=config.final_dataset_dir
        )
        return data_injestion_config
    
    def getPrepareBaseModelConfig(self)->PrepareBaseModelConfig:
        config=self.config.prepare_base_model

        createDir([config.root_dir])
        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            seasonality_mode=str(config.seasonality_mode),
            weekly_seasonality=bool(config.weekly_seasonality),
            daily_seasonality=bool(config.daily_seasonality),
            yearly_seasonality=bool(config.yearly_seasonality)
        )

        return prepare_base_model_config
    
    def getModelTrainingConfig(self)-> ModelTrainingConfig:
        config=self.config.model_train

        createDir([config.root_dir])
        model_train_config=ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            dataset_path=Path(config.dataset_path),
            base_model_path=Path(config.base_model_path),
            store_column=self.params.STORE_COLUMN,
            date_column=self.params.DATE_COLUMN,
            target_column=self.params.TARGET_COLUMN,
            train_split=self.params.TRAIN_SPLIT
        )
        return model_train_config
    
    def getModelEvaluationConfig(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation

        createDir([config.root_dir])
        model_eval_config=ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            evaluation_results_path=Path(config.evaluation_results_path),
            train_split=self.params.TRAIN_SPLIT,
            forecast_periods=self.params.FORECAST_PERIODS,
            frequency=self.params.FREQUENCY,
            dataset_path=Path(config.dataset_path),
            model_path=Path(config.model_path),
            store_column=self.params.STORE_COLUMN,
            date_column=self.params.DATE_COLUMN,
            target_column=self.params.TARGET_COLUMN
        )
        return model_eval_config
    
    def getModelPredConfig(self)->ModelPredictionConfig:
        config=self.config.model_prediction

        createDir([config.root_dir])
        model_pred_config=ModelPredictionConfig(
            root_dir=Path(config.root_dir),
            dataset_path=Path(config.dataset_path),
            model_path=Path(config.model_path)
        )
        return model_pred_config

In [7]:
import zipfile
import gdown
from src.retailDemand import logger
import pandas as pd


In [8]:
import pandas as pd

class DataInjestion:
    def __init__(self,config:DataInjestionConfig):
        self.config=config
    
    def download_file(self):
        try:
            dataset_url=self.config.source_url
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data_injestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix="https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self)->Path:
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            folders = {name.split('/')[0] for name in zip_ref.namelist() if '/' in name}
            logger.info(list(folders)[0])
            logger.info(zip_ref)
        fname=list(folders)[0]
        logger.info(f"Final unzip path: {Path(unzip_path)/fname}")
        return Path(unzip_path)/fname
    
    def dataPrep(self)->pd.DataFrame:
        try:
            final_dir_path=self.extract_zip_file()
            fPath=final_dir_path/"features.csv"
            tPath=final_dir_path/"train.csv"
            sPath=final_dir_path/"stores.csv"

            logger.info("Loading csv files for merging")
            df1=pd.read_csv(tPath)
            df2=pd.read_csv(fPath)
            df3=pd.read_csv(sPath)

            temp_data=df1.merge(df2,on=['Store','Date','IsHoliday'],how='inner')
            final_data=temp_data.merge(df3,on=['Store'],how='inner')

            final_data['Date']=pd.to_datetime(final_data['Date'])

            final_data['Year']=final_data['Date'].dt.year
            final_data['Month']=final_data['Date'].dt.month
            final_data['Week']=final_data['Date'].dt.isocalendar().week
            final_data['Day_Name']=final_data['Date'].dt.day_name
            final_data['Month_Name']=final_data['Date'].dt.month_name

            if 'Weekly_Sales' in final_data.columns and 'Weekly_Sales_x' not in final_data.columns:
                final_data['Weekly_Sales_x'] = final_data['Weekly_Sales']

            logger.info(f"Final merged data shape: {final_data.shape}")
            logger.info(f"Columns: {list(final_data.columns)}")

            return final_data
        except Exception as e:
            logger.error(f"Error loading and merging data: {str(e)}")
            raise

    def save_data(self,name:str="final_data.csv")->Path:
        final_data=self.dataPrep()
        save_dir=self.config.final_dataset_dir
        save_path=Path(save_dir)/name

        final_data.to_csv(save_path,index=False)
        logger.info(f"Final data saved to: {save_path}")
        # logger.info(f"Saved data shape: {self.data.shape}")
        logger.info(f"Ready for model training")

        return save_path

In [9]:
%pwd

'c:\\Users\\rohan\\OneDrive\\Desktop\\New folder\\RetailDemandForecast'

In [ ]:
try:
    config=ConfigurationManager()
    data_injestion_config=config.getDataInjestionConfig()
    data_injestion=DataInjestion(config=data_injestion_config)
    data_injestion.download_file()
    data_injestion.save_data()
except Exception as e:
    raise e

[2025-09-16 20:34:00,156: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 20:34:00,162: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 20:34:00,169: INFO: common: Created directory at: artifacts]
[2025-09-16 20:34:00,173: INFO: common: Created directory at: artifacts/data_injestion]
[2025-09-16 20:34:00,176: INFO: 1062413224: Downloading data from https://drive.google.com/file/d/140EnsVZ6YyG8vDjaLiAm8EvZgzX3bH5B/view?usp=sharing into artifacts/data_injestion/dataset.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=140EnsVZ6YyG8vDjaLiAm8EvZgzX3bH5B
To: c:\Users\rohan\OneDrive\Desktop\New folder\RetailDemandForecast\artifacts\data_injestion\dataset.zip
100%|██████████| 3.27M/3.27M [00:02<00:00, 1.40MB/s]

[2025-09-16 20:34:07,644: INFO: 1062413224: Downloaded data from https://drive.google.com/file/d/140EnsVZ6YyG8vDjaLiAm8EvZgzX3bH5B/view?usp=sharing to artifacts/data_injestion/dataset.zip]


[2025-09-16 20:34:07,810: INFO: 1062413224: dataset1]
[2025-09-16 20:34:07,817: INFO: 1062413224: <zipfile.ZipFile filename='artifacts/data_injestion/dataset.zip' mode='r'>]
[2025-09-16 20:34:07,820: INFO: 1062413224: Final unzip path: artifacts\data_injestion\dataset1]
[2025-09-16 20:34:07,822: INFO: 1062413224: Loading csv files for merging]
[2025-09-16 20:34:08,789: INFO: 1062413224: Final merged data shape: (421570, 22)]
[2025-09-16 20:34:08,789: INFO: 1062413224: Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Year', 'Month', 'Week', 'Day_Name', 'Month_Name', 'Weekly_Sales_x']]
[2025-09-16 20:34:21,084: INFO: 1062413224: Final data saved to: artifacts\data_injestion\final_data.csv]
[2025-09-16 20:34:21,085: INFO: 1062413224: Ready for model training]


In [11]:
from prophet import Prophet
import pickle

class BaseModelPrep:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config=config
        self.model=None
    
    def getBaseModel(self)->Prophet:
        model=Prophet(
            seasonality_mode=self.config.seasonality_mode,
            daily_seasonality=self.config.seasonality_mode,
            weekly_seasonality=self.config.weekly_seasonality,
            yearly_seasonality=self.config.yearly_seasonality
        )
        logger.info("Base model prepared")
        return model
    
    def save_base_model(self):
        model=self.getBaseModel()
        model_path=self.config.base_model_path
        with open(model_path,'wb') as f:
            pickle.dump(model,f)
        logger.info(f"Model saved at path {model_path}")

In [12]:
try:
    config=ConfigurationManager()
    base_model_prep=BaseModelPrep(config.getPrepareBaseModelConfig())
    base_model_prep.save_base_model()
except Exception as e:
    raise e

[2025-09-16 20:34:22,481: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 20:34:22,487: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 20:34:22,489: INFO: common: Created directory at: artifacts]
[2025-09-16 20:34:22,492: INFO: common: Created directory at: artifacts/prepare_base_model]
[2025-09-16 20:34:22,587: DEBUG: command: cmd: where.exe tbb.dll
cwd: None]
[2025-09-16 20:34:22,661: DEBUG: model: Adding TBB (c:\Users\rohan\.conda\envs\tensorflow\Lib\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH]
[2025-09-16 20:34:22,664: INFO: 1068067062: Base model prepared]
[2025-09-16 20:34:22,666: INFO: 1068067062: Model saved at path artifacts\prepare_base_model\base_model.pkl]


In [13]:
class ModelTrain:
    def __init__(self,config:ModelTrainingConfig):
        self.config=config
        self.dataset=None
        self.models={}
        self.base_params=None
    
    def load_dataset(self):
        df_path=self.config.dataset_path
        df=pd.read_csv(df_path)
        self.dataset=df
        logger.info("Dataset loaded for model tarining")

    def load_base_model_params(self):
        if self.base_params is not None:
            return self.base_params

        base_model_path=self.config.base_model_path

        if not Path(base_model_path).exists():
            raise FileNotFoundError(f"No base model at {base_model_path}")
        
        with open(base_model_path,'rb') as f:
            base_model=pickle.load(f)
        logger.info("Base model loaded successfully")
    
        # Debug: Log base model attributes
        logger.info(f"Base model type: {type(base_model)}")

        self.base_params = {
        'growth': str(getattr(base_model, 'growth', 'linear')),
        'seasonality_mode': str(getattr(base_model, 'seasonality_mode', 'multiplicative')),
        'daily_seasonality': bool(getattr(base_model, 'daily_seasonality', False)),
        'weekly_seasonality': bool(getattr(base_model, 'weekly_seasonality', True)),
        'yearly_seasonality': bool(getattr(base_model, 'yearly_seasonality', True)),
        'changepoint_prior_scale': float(getattr(base_model, 'changepoint_prior_scale', 0.05)),
        'seasonality_prior_scale': float(getattr(base_model, 'seasonality_prior_scale', 10.0)),
        'holidays_prior_scale': float(getattr(base_model, 'holidays_prior_scale', 10.0)),
        'interval_width': float(getattr(base_model, 'interval_width', 0.80)),
        'mcmc_samples': int(getattr(base_model, 'mcmc_samples', 0)),
        'uncertainty_samples': int(getattr(base_model, 'uncertainty_samples', 1000))
    }
        if self.base_params['seasonality_mode'] not in ['additive', 'multiplicative']:
            logger.warning(f"Invalid seasonality_mode: {self.base_params['seasonality_mode']}, using 'multiplicative'")
            self.base_params['seasonality_mode'] = 'multiplicative'
        
        logger.info("Extracted base model parameters:")
        for key, value in self.base_params.items():
            logger.info(f"  {key} ({type(value).__name__}): {value}")
    
    def create_new_model_instance(self)->Prophet:
        self.load_base_model_params()
        model=Prophet(**self.base_params)
        logger.info(f"Created base model's new instance")
        return model

    def store_data_prep(self,store_id:int)->pd.DataFrame:
        store_data=self.dataset[self.dataset[self.config.store_column]==store_id]

        store_data=store_data[[self.config.date_column,self.config.target_column]]
        store_data=store_data.groupby(self.config.date_column).sum()
        store_data.reset_index(inplace=True)
        store_data.columns=['ds','y']
        logger.info(f"Data prepared for store {store_id}")
        return store_data
    
    def train_store(self,store_id:int)->Prophet:
        logger.info(f"Started training for store {store_id}")
        data=self.store_data_prep(store_id=store_id)
        train_data=data.iloc[:self.config.train_split]
        model=self.create_new_model_instance()
        model.fit(train_data)
        self.models[store_id]=model
        logger.info(f"Model trained for store {store_id}")
        return model
    
    def trainAll(self):
        stores=self.dataset[self.config.store_column].unique()
        logger.info(f"Starting model training for {len(stores)} stores")

        for store_id in stores:
            #store_data=self.store_data_prep(store_id)
            self.train_store(store_id)
            logger.info(f"Model trained for store {store_id}")
    
    def save_models(self):
        save_dir = Path(self.config.trained_model_path)
        save_dir.mkdir(parents=True, exist_ok=True)
        for store_id,model in self.models.items():
            model_path=self.config.trained_model_path/f"prophet_{store_id}.pkl"
            with open(model_path,'wb') as f:
                pickle.dump(model,f)
            logger.info(f"model saved for store {store_id} at {model_path}")

In [14]:
try:
    config=ConfigurationManager()
    train_model=ModelTrain(config.getModelTrainingConfig())
    train_model.load_dataset()
    train_model.trainAll()
    train_model.save_models()
except Exception as e:
    raise e

[2025-09-16 20:34:22,716: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 20:34:22,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 20:34:22,725: INFO: common: Created directory at: artifacts]
[2025-09-16 20:34:22,727: INFO: common: Created directory at: artifacts/model_train]
[2025-09-16 20:34:25,120: INFO: 465351063: Dataset loaded for model tarining]
[2025-09-16 20:34:25,138: INFO: 465351063: Starting model training for 45 stores]
[2025-09-16 20:34:25,138: INFO: 465351063: Started training for store 1]
[2025-09-16 20:34:25,152: INFO: 465351063: Data prepared for store 1]
[2025-09-16 20:34:25,156: INFO: 465351063: Base model loaded successfully]
[2025-09-16 20:34:25,158: INFO: 465351063: Base model type: <class 'prophet.forecaster.Prophet'>]
[2025-09-16 20:34:25,159: INFO: 465351063: Extracted base model parameters:]
[2025-09-16 20:34:25,161: INFO: 465351063:   growth (str): linear]
[2025-09-16 20:34:25,161: INFO: 465351063:   se

20:34:25 - cmdstanpy - INFO - Chain [1] start processing


[2025-09-16 20:34:25,397: INFO: model: Chain [1] start processing]


20:34:25 - cmdstanpy - INFO - Chain [1] done processing


[2025-09-16 20:34:25,462: INFO: model: Chain [1] done processing]
[2025-09-16 20:34:25,480: INFO: 465351063: Model trained for store 1]
[2025-09-16 20:34:25,481: INFO: 465351063: Model trained for store 1]
[2025-09-16 20:34:25,482: INFO: 465351063: Started training for store 2]
[2025-09-16 20:34:25,490: INFO: 465351063: Data prepared for store 2]
[2025-09-16 20:34:25,494: DEBUG: command: cmd: where.exe tbb.dll
cwd: None]
[2025-09-16 20:34:25,559: DEBUG: model: TBB already found in load path]
[2025-09-16 20:34:25,559: INFO: 465351063: Created base model's new instance]
[2025-09-16 20:34:25,577: DEBUG: filesystem: input tempfile: C:\Users\rohan\AppData\Local\Temp\tmpsnym8bm2\a_9jvg2g.json]
[2025-09-16 20:34:25,598: DEBUG: filesystem: input tempfile: C:\Users\rohan\AppData\Local\Temp\tmpsnym8bm2\rvjm3lqf.json]
[2025-09-16 20:34:25,601: DEBUG: model: idx 0]
[2025-09-16 20:34:25,601: DEBUG: model: running CmdStan, num_threads: None]
[2025-09-16 20:34:25,604: DEBUG: model: CmdStan args: ['C:

...

[2025-09-16 20:34:34,046: INFO: model: Chain [1] done processing]
[2025-09-16 20:34:34,064: INFO: 465351063: Model trained for store 44]
[2025-09-16 20:34:34,064: INFO: 465351063: Model trained for store 44]
[2025-09-16 20:34:34,064: INFO: 465351063: Started training for store 45]
[2025-09-16 20:34:34,073: INFO: 465351063: Data prepared for store 45]
[2025-09-16 20:34:34,076: DEBUG: command: cmd: where.exe tbb.dll
cwd: None]
[2025-09-16 20:34:34,142: DEBUG: model: TBB already found in load path]
[2025-09-16 20:34:34,143: INFO: 465351063: Created base model's new instance]
[2025-09-16 20:34:34,171: DEBUG: filesystem: input tempfile: C:\Users\rohan\AppData\Local\Temp\tmpsnym8bm2\x1vuymqo.json]
[2025-09-16 20:34:34,189: DEBUG: filesystem: input tempfile: C:\Users\rohan\AppData\Local\Temp\tmpsnym8bm2\_uw57jf_.json]
[2025-09-16 20:34:34,192: DEBUG: model: idx 0]
[2025-09-16 20:34:34,192: DEBUG: model: running CmdStan, num_threads: None]
[2025-09-16 20:34:34,192: DEBUG: model: CmdStan args: 

20:34:34 - cmdstanpy - INFO - Chain [1] start processing


[2025-09-16 20:34:34,195: INFO: model: Chain [1] start processing]


20:34:34 - cmdstanpy - INFO - Chain [1] done processing


[2025-09-16 20:34:34,260: INFO: model: Chain [1] done processing]
[2025-09-16 20:34:34,275: INFO: 465351063: Model trained for store 45]
[2025-09-16 20:34:34,276: INFO: 465351063: Model trained for store 45]
[2025-09-16 20:34:34,282: INFO: 465351063: model saved for store 1 at artifacts\model_train\trained_models\prophet_1.pkl]
[2025-09-16 20:34:34,284: INFO: 465351063: model saved for store 2 at artifacts\model_train\trained_models\prophet_2.pkl]
[2025-09-16 20:34:34,288: INFO: 465351063: model saved for store 3 at artifacts\model_train\trained_models\prophet_3.pkl]
[2025-09-16 20:34:34,291: INFO: 465351063: model saved for store 4 at artifacts\model_train\trained_models\prophet_4.pkl]
[2025-09-16 20:34:34,293: INFO: 465351063: model saved for store 5 at artifacts\model_train\trained_models\prophet_5.pkl]
[2025-09-16 20:34:34,297: INFO: 465351063: model saved for store 6 at artifacts\model_train\trained_models\prophet_6.pkl]
[2025-09-16 20:34:34,301: INFO: 465351063: model saved for s

In [15]:
%pwd
import pickle
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [16]:
from typing import Dict


class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config
        self.models=None
        self.forecasts={}
        self.dataset=None
        self.results={}

    def load_models(self)->dict:
        models={}
        models_path=self.config.model_path
        if not models_path.exists():
            logger.error("No model path exists")
        
        model_files=list(models_path.glob("prophet_*.pkl"))

        if not model_files:
            logger.warning("No models found")
        logger.info(f"Found {len(model_files)} models")

        for model_file in model_files:
            file=model_file.stem
            store=int(file.split("_")[1])
            with open(model_file,'rb') as f:
                model=pickle.load(f)
            models[store]=model
            logger.info(f"Model loaded for store {store}")
        self.models=models

    def load_dataset(self):
        df_path=self.config.dataset_path
        df=pd.read_csv(df_path)
        self.dataset=df
        logger.info("Dataset loaded for model tarining")

    def store_data_prep(self,store_id:int)->pd.DataFrame:
        store_data=self.dataset[self.dataset[self.config.store_column]==store_id]

        store_data=store_data[[self.config.date_column,self.config.target_column]]
        store_data=store_data.groupby(self.config.date_column).sum()
        store_data.reset_index(inplace=True)
        store_data.columns=['ds','y']
        logger.info(f"Data prepared for store {store_id}")
        return store_data
    
    def predictStore(self,store_id:int)-> pd.DataFrame:
        model=self.models[store_id]

        future=model.make_future_dataframe(
            periods=self.config.forecast_periods,
            freq=self.config.frequency
        )
        forecast=model.predict(future)
        self.forecasts[store_id]=forecast

        logger.info(f"Forecasts generated for store {store_id}")
        return forecast
    
    def modelEval(self,store_id:int)->Dict[str,float]:
        store_data=self.store_data_prep(store_id)
        testData=store_data.iloc[self.config.train_split:]
        
        if store_id not in self.forecasts:
            raise ValueError(f"No store found for id= {store_id}")
        
        forecast=self.forecasts[store_id]
        y_pred=forecast['yhat'].tail(len(testData)).values
        y_true=testData['y'].values

        mape=mean_absolute_percentage_error(y_true=y_true,y_pred=y_pred)*100
        mae=mean_absolute_error(y_true,y_pred)

        metrics = {
            'MAPE': round(mape, 4),
            'MAE': round(mae, 4)
        }

        self.results[store_id] = metrics
        logger.info(f"Evaluation completed for store {store_id}: MAPE={mape:.2f}%")

        return metrics
    
    def evalAll(self):
        stores=self.dataset[self.config.store_column].unique()
        
        logger.info(f"Starting training {len(stores)} stores")

        for store_id in stores:
            self.predictStore(store_id)
            self.modelEval(store_id)

    def save_evaluation_results(self):
        if not self.results:
            logger.warning("No results to save")
            return
        
        results_df = pd.DataFrame.from_dict(self.results, orient='index')
        results_df.reset_index(inplace=True)
        results_df.columns = ['Store', 'MAPE', 'MAE']
        
        results_path = self.config.evaluation_results_path
        results_path.parent.mkdir(parents=True, exist_ok=True)
        results_df.to_csv(results_path, index=False)
        
        logger.info(f"Evaluation Results Summary:")
        logger.info(f"  Average MAPE: {results_df['MAPE'].mean():.2f}%")
        logger.info(f"  Median MAPE: {results_df['MAPE'].median():.2f}%")
        logger.info(f"  Best MAPE: {results_df['MAPE'].min():.2f}% (Store {results_df.loc[results_df['MAPE'].idxmin(), 'Store']})")
        logger.info(f"  Results saved to: {results_path}")
        
        return results_df


In [17]:
try:
    config=ConfigurationManager()
    model_eval=ModelEvaluation(config.getModelEvaluationConfig())
    model_eval.load_models()
    model_eval.load_dataset()
    model_eval.evalAll()
    model_eval.save_evaluation_results()
except Exception as e:
    raise e

[2025-09-16 20:34:36,823: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 20:34:36,825: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 20:34:36,825: INFO: common: Created directory at: artifacts]
[2025-09-16 20:34:36,830: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-09-16 20:34:36,832: INFO: 850444778: Found 45 models]
[2025-09-16 20:34:36,848: INFO: 850444778: Model loaded for store 1]
[2025-09-16 20:34:36,863: INFO: 850444778: Model loaded for store 10]
[2025-09-16 20:34:36,876: INFO: 850444778: Model loaded for store 11]
[2025-09-16 20:34:36,877: INFO: 850444778: Model loaded for store 12]
[2025-09-16 20:34:36,902: INFO: 850444778: Model loaded for store 13]
[2025-09-16 20:34:36,922: INFO: 850444778: Model loaded for store 14]
[2025-09-16 20:34:36,936: INFO: 850444778: Model loaded for store 15]
[2025-09-16 20:34:36,940: INFO: 850444778: Model loaded for store 16]
[2025-09-16 20:34:36,960: INFO: 850444778: 

In [20]:
from typing import Any


class ModelPredictor:
    def __init__(self,config:ModelPredictionConfig):
        self.config=config
        self.models={}

    def load_models(self)->dict:
        models={}
        model_path=self.config.model_path
        if not model_path.exists():
            logger.error("No model path exists")
        
        model_files=list(model_path.glob("prophet_*.pkl"))

        if not model_files:
            logger.warning("No models found")
        logger.info(f"Found {len(model_files)} models")

        for model_file in model_files:
            file=model_file.stem
            store=int(file.split("_")[1])
            with open(model_file,'rb') as f:
                model=pickle.load(f)
            models[store]=model
            logger.info(f"Model loaded for store {store}")
        self.models=models

    def predSales(self,store_id:int,date:str)->Dict[str,Any]:
        if store_id not in self.models:
            raise ValueError(f"Model for store {store_id} not found.")
        
        model=self.models[store_id]
        future=pd.DataFrame({'ds':[pd.to_datetime(date)]})
        forecast=model.predict(future)

        res={
            "store_id":store_id,
            "prediction_date":date,
            "predicted_sales":round(forecast['yhat'].iloc[0],2),
            'lower_bound': round(forecast['yhat_lower'].iloc[0], 2),
            'upper_bound': round(forecast['yhat_upper'].iloc[0], 2)
        }

        logger.info(f"prediction for store {store_id} on {date} is ${res['predicted_sales']:,.2f}")
        return res

In [21]:
try:
    config=ConfigurationManager()
    model_pred=ModelPredictor(config.getModelPredConfig())
    model_pred.load_models()
    model_pred.predSales(store_id=3,date="2010-02-05")
except Exception as e:
    raise e

[2025-09-16 20:35:36,501: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 20:35:36,508: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 20:35:36,511: INFO: common: Created directory at: artifacts]
[2025-09-16 20:35:36,513: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-09-16 20:35:36,518: INFO: 1353086594: Found 45 models]
[2025-09-16 20:35:36,524: INFO: 1353086594: Model loaded for store 1]
[2025-09-16 20:35:36,528: INFO: 1353086594: Model loaded for store 10]
[2025-09-16 20:35:36,535: INFO: 1353086594: Model loaded for store 11]
[2025-09-16 20:35:36,543: INFO: 1353086594: Model loaded for store 12]
[2025-09-16 20:35:36,549: INFO: 1353086594: Model loaded for store 13]
[2025-09-16 20:35:36,553: INFO: 1353086594: Model loaded for store 14]
[2025-09-16 20:35:36,558: INFO: 1353086594: Model loaded for store 15]
[2025-09-16 20:35:36,564: INFO: 1353086594: Model loaded for store 16]
[2025-09-16 20:35:36,572: INFO: 13